In [189]:
import json
import requests

In [199]:
url = 'http://127.0.0.1:8000/tasksJSON/'
f = requests.get(url)
json_data = f.json()

In [200]:
class Lane:
    def __init__(self, name, length):
        self.id = name
        self.length = length
        self.jobs = {}
        self.free_spaces = [(0,self.length)]
        self.remaining = self.length
    
    def findFit(self, win_start, win_end, job_len):
        position = (win_start, win_start+job_len)
        fits = []
        space = (0,0)
        for s in self.free_spaces:
            not_done = True
            if position[1]<s[0]:
                continue
                
            if position[0]>s[1]:
                continue
            
            while not_done:               
                if s[0] <= position[0] and s[1] >= position[1]:
                    not_done = False
                    fits.append(position)
                    space = s

                elif position[1] == win_end:
                    not_done = False 
                    position = (win_start, win_start+job_len)
                    
                else:    
                    position += (1,1)
           
        if len(fits) == 0:
            return -1 
        
        else:
            return fits[0], space
                    
    
    # splits the empty space in the lane into seperate pieces 
    def splitLane(self, _start, _end, space):
        if space[0] != _start:
            self.free_spaces.append((space[0], _start))
            
        if space[1] != _end: 
            self.free_spaces.append((_end, space[1]))
            
        self.free_spaces.remove(space)
        
    
    def add_job(self, job):
        try:
            (_space, _old) = self.findFit(job['least_start'], job['max_end'], job['length'])
            _start = _space[0]
            _end = _start + job['length']
            self.jobs[job['id']] = {'Position': (_start, _end)}
            self.splitLane(_start, _end, _old)
            print("Job: %s" % job['id'])
            print("Job Length: %s" % job['length'])
            print("Job Window: %s" % job['least_start'], job['max_end'])
            print("Job Placed between: %s" % _start, _end)
            print("Free Spaces: %s" % self.free_spaces)
            print("_______________________________________________________________________")
        except TypeError:
            return -1
        
        self.remaining -= job['length']

        
        
    def __eq__(self,other):
        return self.remaining == other.remaining
    
    def __lt__(self,other):
        return self.remaining < other.remaining
    
    def __gt__(self,other):
        return self.remaining > other.remaining
        
    def __str__(self):
        return '(jobs=%s, remaining=%s)' % (str(self.jobs), self.remaining)

In [201]:
class laneList(object):
    def __init__(self, lane_size):
        self.lane_size = lane_size
        self.lanes = []
        self.numLanes = 0
        self.extra = 0
        
        
    def add_lane(self):
        self.lanes.append(Lane(self.numLanes+1, self.lane_size))
        self.numLanes = len(self.lanes)
        self.lanes = sorted(self.lanes)
        
        
    def get_extra(self):
        self.extra = 0
        for i in range(self.numLanes):
            self.extra += self.lanes[i].remaining
          
        
    def FF(self, job):
        current = 0
        not_done = True
        
        while not_done: 
            if len(self.lanes) == 0:
                self.add_lane()
            
            try:
                if self.lanes[current].add_job(job) != -1:
                    not_done = False 
                    self.get_extra()
                    break
                    return 0 

                else:
                    current += 1
                    
            except IndexError:
                self.add_lane()
            
    
    def __str__(self):
        if self.numLanes != 0:
            for lane in self.lanes:
                print(lane)
            return 'Number of Lanes: %s, Free Space: %s' %(self.numLanes, self.extra)
        else: 
            return 'No Lanes'

In [202]:
jobs = []
fixed_jobs = []
for task in json_data: 
    if task['fixed'] == True:
        fixed_jobs.append(task)
    else: 
        jobs.append(task)

In [203]:
lanes = laneList(100)

In [204]:
for job in fixed_jobs:
    lanes.FF(job)
for job in jobs: 
    lanes.FF(job)
print(lanes)

Job: 103
Job Length: 40
Job Window: 0 40
Job Placed between: 0 40
Free Spaces: [(40, 100)]
_______________________________________________________________________
Job: 104
Job Length: 15
Job Window: 40 45
Job Placed between: 40 55
Free Spaces: [(55, 100)]
_______________________________________________________________________
Job: 105
Job Length: 15
Job Window: 55 70
Job Placed between: 55 70
Free Spaces: [(70, 100)]
_______________________________________________________________________
Job: 106
Job Length: 15
Job Window: 0 15
Job Placed between: 0 15
Free Spaces: [(15, 100)]
_______________________________________________________________________
(jobs={103: {'Position': (0, 40)}, 104: {'Position': (40, 55)}, 105: {'Position': (55, 70)}}, remaining=30)
(jobs={106: {'Position': (0, 15)}}, remaining=85)
Number of Lanes: 2, Free Space: 115


In [196]:
lane_list = {}
for lane in lanes.lanes:
    lane_list[lane.id] = lane.jobs
    print(lane.jobs)

{99: {'Position': (0, 40)}, 101: {'Position': (55, 70)}}
{100: {'Position': (30, 45)}, 102: {'Position': (0, 15)}}


In [197]:
lane_list

{1: {99: {'Position': (0, 40)}, 101: {'Position': (55, 70)}},
 2: {100: {'Position': (30, 45)}, 102: {'Position': (0, 15)}}}

In [198]:
print(json.dumps(lane_list, indent=2))

{
  "1": {
    "99": {
      "Position": [
        0,
        40
      ]
    },
    "101": {
      "Position": [
        55,
        70
      ]
    }
  },
  "2": {
    "100": {
      "Position": [
        30,
        45
      ]
    },
    "102": {
      "Position": [
        0,
        15
      ]
    }
  }
}


In [102]:
with open('LaneData.json','w') as f: 
    json.dump(lane_list,f)

In [155]:
url = 'http://127.0.0.1:8000/addPosition/'
f = requests.get(url)

cookies = {'csrftoken':'7k3payN2kvwfRNIC6J4o244UHugF1DImabZq8Xp44mLXCZ75Re63Rzdtf1jewvvL'}

In [188]:
for key, values in lane_list.items():
    for k, v in lane_list[key].items():
        start = v[0]
        end = v[1]
        post_url = url + 'addPosition/' + str(key) + '/' + str(k) + '/' + str(start) + '/' + str(end)
        print(post_url)

http://127.0.0.1:8000/addPosition/1/47/0/40
http://127.0.0.1:8000/addPosition/1/49/55/70
http://127.0.0.1:8000/addPosition/2/48/30/100
http://127.0.0.1:8000/addPosition/2/50/0/15


In [186]:
url = 'http://127.0.0.1:8000/'
client = requests.session()
client.get(url)

if 'csrftoken' in client.cookies:
    csrftoken = client.cookies['csrftoken']
    print(csrftoken)

data = dict(csrfmiddlewaretoken=csrftoken)

for key, values in lane_list.items():
    for k, v in lane_list[key].items():
        client = requests.session()
        client.get(url)

        if 'csrftoken' in client.cookies:
            csrftoken = client.cookies['csrftoken']
            print(csrftoken)

        data = dict(csrfmiddlewaretoken=csrftoken)

        post_url = url + 'addPosition/' + str(key) + '/' + str(k) + '/' + str(v[0]) + '/' + str(v[1])
        client.post(post_url, data=data)

eev0ebWKNYwLtzasBt8mk9JTtmdnLGqxtoojJSWh1XqXtY09eOtYtTGtzto7lFLB
HWyJ0ZpnaFJTWhtx8CIQ0w24lvnUl6kxFchmt5my2MsUk7n6g3WHtXnHwuZshAis
psNd63K9DgO7tixgsEZhsr0MtegOuge1rIgL6tVfDpwu2okX0K5VEyP87Wd5MU3w
9bGX16ud4YkZI1L03S1jU2UMscOw2f094aXvBEx4aaQtCuMpcAq6JaLrn9T2YFYK
JEuTbzKt5m7OIVmzRmAR58OX1Webu4B5w6M3B1I9uc4aAWtMwj9vJvwLNIxHikjS
